In [1]:
# -*- coding: utf-8 -*-
"""
Created on April 3, 2024

@author:  Narges Yarahmadi Gharaei
"""

'\nCreated on April 3, 2024\n\n@author:  Narges Yarahmadi Gharaei\n'

In [2]:

from __future__ import absolute_import
from __future__ import print_function
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Input,
    Activation,
    Concatenate,
    concatenate,
    Dropout,
    Reshape,
    Permute,
    Dense,
    UpSampling2D,
    Flatten
    )

from tensorflow.keras.optimizers import SGD, RMSprop , Adam
from tensorflow.keras.layers import Conv2D, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model

weight_decay = 1e-5
K.set_image_data_format('channels_last')

In [ ]:
def _conv_bn_relu(nb_filter, row, col, strides = (1,1)):
    def f(input):
        conv_a = Conv2D(nb_filter, (row, col), strides = strides,
                               kernel_initializer = 'orthogonal',
                               padding='same', use_bias = False)(input)
        norm_a = BatchNormalization()(conv_a)
        act_a = Activation(activation = 'relu')(norm_a)
        return act_a
    return f

In [ ]:
def _conv_bn_relu_x2(nb_filter, row, col, strides = (1,1)):
    def f(input):
        conv_a = Conv2D(nb_filter, (row, col), strides = strides,
                               kernel_initializer = 'orthogonal', padding = 'same',use_bias = False,
                               kernel_regularizer = l2(weight_decay),
                               bias_regularizer = l2(weight_decay))(input)
        norm_a = BatchNormalization()(conv_a)
        act_a = Activation(activation = 'relu')(norm_a)
        conv_b = Conv2D(nb_filter, (row, col), strides = strides,
                               kernel_initializer = 'orthogonal', padding = 'same',use_bias = False,
                               kernel_regularizer = l2(weight_decay),
                               bias_regularizer = l2(weight_decay))(act_a)
        norm_b = BatchNormalization()(conv_b)
        act_b = Activation(activation = 'relu')(norm_b)
        return act_b
    return f

In [ ]:
def FCRN_A_base(input):
    block1 = _conv_bn_relu(32,3,3)(input)
    pool1 = MaxPool2D(pool_size=(2,2))(block1)
    # =========================================================================
    block2 = _conv_bn_relu(64,3,3)(pool1)
    pool2 = MaxPool2D(pool_size=(2, 2))(block2)
    # =========================================================================
    block3 = _conv_bn_relu(128,3,3)(pool2)
    pool3 = MaxPool2D(pool_size=(2, 2))(block3)
    # =========================================================================
    block4 = _conv_bn_relu(512,3,3)(pool3)
    # =========================================================================
    up5 = UpSampling2D(size=(2, 2))(block4)
    block5 = _conv_bn_relu(128,3,3)(up5)
    # =========================================================================
    up6 = UpSampling2D(size=(2, 2))(block5)
    block6 = _conv_bn_relu(64,3,3)(up6)
    # =========================================================================
    up7 = UpSampling2D(size=(2, 2))(block6)
    block7 = _conv_bn_relu(32,3,3)(up7)
    return block7

In [ ]:
def FCRN_A_base_v2(input):
    block1 = _conv_bn_relu_x2(32,3,3)(input)
    pool1 = MaxPool2D(pool_size=(2,2))(block1)
    # =========================================================================
    block2 = _conv_bn_relu_x2(64,3,3)(pool1)
    pool2 = MaxPool2D(pool_size=(2, 2))(block2)
    # =========================================================================
    block3 = _conv_bn_relu_x2(128,3,3)(pool2)
    pool3 = MaxPool2D(pool_size=(2, 2))(block3)
    # =========================================================================
    block4 = _conv_bn_relu(512,3,3)(pool3)
    # =========================================================================
    up5 = UpSampling2D(size=(2, 2))(block4)
    block5 = _conv_bn_relu_x2(128,3,3)(up5)
    # =========================================================================
    up6 = UpSampling2D(size=(2, 2))(block5)
    block6 = _conv_bn_relu_x2(64,3,3)(up6)
    # =========================================================================
    up7 = UpSampling2D(size=(2, 2))(block6)
    block7 = _conv_bn_relu_x2(32,3,3)(up7)
    return block7

In [ ]:
def U_net_base(input, nb_filter = 64):
    block1 = _conv_bn_relu_x2(nb_filter,3,3)(input)
    pool1 = MaxPool2D(pool_size=(2,2))(block1)
    # =========================================================================
    block2 = _conv_bn_relu_x2(nb_filter,3,3)(pool1)
    pool2 = MaxPool2D(pool_size=(2, 2))(block2)
    # =========================================================================
    block3 = _conv_bn_relu_x2(nb_filter,3,3)(pool2)
    pool3 = MaxPool2D(pool_size=(2, 2))(block3)
    # =========================================================================
    block4 = _conv_bn_relu_x2(nb_filter,3,3)(pool3)
    up4 = concatenate([UpSampling2D(size=(2, 2))(block4), block3], axis=-1)
    # =========================================================================
    block5 = _conv_bn_relu_x2(nb_filter,3,3)(up4)
    up5 = concatenate([UpSampling2D(size=(2, 2))(block5), block2], axis=-1)
    # =========================================================================
    block6 = _conv_bn_relu_x2(nb_filter,3,3)(up5)
    up6 = concatenate([UpSampling2D(size=(2, 2))(block6), block1], axis=-1)
    # =========================================================================
    block7 = _conv_bn_relu(nb_filter,3,3)(up6)
    return block7

In [ ]:
def buildModel_FCRN_A (input_dim):
    input_ = Input (shape = (input_dim))
    # =========================================================================
    act_ = FCRN_A_base (input_)
    # =========================================================================
    density_pred =  Conv2D(1, 1, 1, use_bias = False, activation='linear',\
                                  kernel_initializer='orthogonal',name='pred',padding='same')(act_)
    # =========================================================================
    model = Model (inputs = input_, outputs = density_pred)
    opt = SGD(lr = 1e-2, momentum = 0.9, nesterov = True)
    model.compile(optimizer = opt, loss = 'mse')
    return model

In [ ]:
def buildModel_FCRN_A_v2 (input_dim):
    input_ = Input (shape = (input_dim))
    # =========================================================================
    act_ = FCRN_A_base_v2 (input_)
    # =========================================================================
    density_pred =  Conv2D(1, 1, 1, use_bias = False, activation='linear',\
                                  kernel_initializer='orthogonal',name='pred',padding='same')(act_)
    # =========================================================================
    model = Model (inputs = input_, outputs = density_pred)
    opt = SGD(lr = 1e-2, momentum = 0.9, nesterov = True)
    model.compile(optimizer = opt, loss = 'mse')
    return model

In [1]:
def buildModel_Light_U_net (input_dim):
    input_ = Input (shape = (input_dim))
    # =========================================================================
    act_ = U_net_base (input_, nb_filter = 64 )
    # =========================================================================
    density_pred =  Conv2D(1, 1, 1, use_bias = False, activation='linear',\
                                  kernel_initializer='orthogonal',name='pred',padding='same')(act_)
    # =========================================================================
    model = Model (inputs = input_, outputs = density_pred)
    opt = RMSprop(1e-3)
    model.compile(optimizer = opt, loss = 'mse', run_eagerly=False)
    return model


In [ ]:
""" def buildModelCNN(img_shape):
    # CNN Architecture:
    # 3 convolutional layers, each has different number of feature maps
    #l2 = keras.regularizers.l2(1e-5)
    l2 = None
    inputs = keras.Input(shape=img_shape)
    # take initial image with 3 channels and produce one with 16
    # padding is same since we don't want image to become smaller as it propagates through the layers
    conv1 = keras.layers.Conv2D(16,5,padding='same',activation='relu',kernel_initializer='glorot_normal',kernel_regularizer=l2)(inputs)
    conv1 = keras.layers.BatchNormalization(momentum=0.99)(conv1)
    # 2nd layer has 32 feature maps
    conv2 = keras.layers.Conv2D(32,5,padding='same',activation='relu',kernel_initializer='glorot_normal',kernel_regularizer=l2)(conv1)
    conv2 = keras.layers.BatchNormalization(momentum=0.99)(conv2)
    # in last layer we bring feature maps back to 1, corresponding to a single black & white channel
    outputs = keras.layers.Conv2D(1,5,padding='same',activation='relu',kernel_initializer='glorot_normal',kernel_regularizer=l2)(conv2)
    model = keras.Model(inputs=inputs, outputs=outputs)
    opt = keras.optimizers.Adam(learning_rate=1e-4)
    #model.compile(optimizer=opt,loss=loss_function)
    model.compile(optimizer=opt,loss="categorical_crossentropy")
    return model """

In [3]:


# def build_unet_model(input_dim):
#     inputs = Input(input_dim)

#     # Contracting Path
#     c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
#     c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
#     p1 = MaxPool2D((2, 2))(c1)

#     c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
#     c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
#     p2 = MaxPool2D((2, 2))(c2)

#     c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
#     c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
#     p3 = MaxPool2D((2, 2))(c3)

#     c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
#     c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
#     p4 = MaxPool2D(pool_size=(2, 2))(c4)

#     # Bottleneck
#     c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
#     c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

#     # Expansive Path
#     u6 = UpSampling2D((2, 2))(c5)
#     u6 = concatenate([u6, c4])
#     c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
#     c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

#     u7 = UpSampling2D((2, 2))(c6)
#     u7 = concatenate([u7, c3])
#     c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
#     c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

#     u8 = UpSampling2D((2, 2))(c7)
#     u8 = concatenate([u8, c2])
#     c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
#     c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

#     u9 = UpSampling2D((2, 2))(c8)
#     u9 = concatenate([u9, c1])
#     c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
#     c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

#     outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

#     model = Model(inputs=[inputs], outputs=[outputs])
#     # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
#     opt = SGD(lr = 1e-2, momentum = 0.9, nesterov = True)
#     model.compile(optimizer = opt, loss = 'mse')
#     return model

In [ ]:
def build_unet_model(input_dim):
    inputs = Input(input_dim)

    # Contracting Path with Batch Normalization
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    c3 = BatchNormalization()(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    c4 = BatchNormalization()(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    # Bottleneck with Dropout
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Dropout(0.5)(c5)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    c5 = Dropout(0.5)(c5)

    # Expansive Path
    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    c6 = BatchNormalization()(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    c7 = BatchNormalization()(c7)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    c8 = BatchNormalization()(c8)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
    c9 = BatchNormalization()(c9)

    # outputs = Conv2D(1, 1, 1), activation='linear')(c9)

    # model = Model(inputs=[inputs], outputs=[outputs])
    # opt = SGD(lr = 1e-2, momentum = 0.9, nesterov = True)
    # model.compile(optimizer = opt, loss = 'mse')


    density_pred =  Conv2D(1, 1, 1, use_bias = False, activation='linear',\
                                  kernel_initializer='orthogonal',name='pred',padding='same')(c9)
    # =========================================================================
    model = Model (inputs = inputs, outputs = density_pred)
    opt = RMSprop(1e-3)
    model.compile(optimizer = opt, loss = 'mse', run_eagerly=False)
    return model

In [ ]:

def build_unet_model2(input_dim):
    inputs = Input(input_dim)

    # Contracting Path with Batch Normalization
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    # (Other layers remain the same)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    c3 = BatchNormalization()(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    c4 = BatchNormalization()(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    # Bottleneck with Dropout
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Dropout(0.5)(c5)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    c5 = Dropout(0.5)(c5)

    # Expansive Path
    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    c6 = BatchNormalization()(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    c7 = BatchNormalization()(c7)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    c8 = BatchNormalization()(c8)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
    c9 = BatchNormalization()(c9)

    # outputs = Conv2D(1, (1, 1), activation='linear')(c9)

    # model = Model(inputs=[inputs], outputs=[outputs])
    # opt = Adam(lr=1e-3)  # Using Adam optimizer



    density_pred =  Conv2D(1, 1, 1, use_bias = False, activation='linear',\
                                  kernel_initializer='orthogonal',name='pred',padding='same')(c9)
    # =========================================================================
    model = Model (inputs = inputs, outputs = density_pred)
    opt = RMSprop(1e-3)
    model.compile(optimizer = opt, loss = 'mse', run_eagerly=False)

    
    return model